# Seq2Seq

이름 : 박종락

기수 : 11기

작성자 : 10 기 신재우

In [74]:
# colab 환경에서 학습을 진행하실 분들은 구글드라이브를 연동해주세요
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 시작 전 실행할 것들

In [ ]:
%pip install konlpy

In [ ]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
import pandas as pd
import numpy as np
import tqdm
import spacy
import torchtext
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import random
import os
from ast import literal_eval
import torchtext.vocab as vocab
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

### 경로 설정

In [75]:
path_to_folder = "/content/drive/MyDrive/DSL/[0225]RNN + Transformers_ParkJongRak"

In [76]:
path_train = os.path.join(path_to_folder, "Dataset/train_dataset_Tokenized.csv")
path_val = os.path.join(path_to_folder, "Dataset/val_dataset_Tokenized.csv")
path_test = os.path.join(path_to_folder, "Dataset/test_dataset_Tokenized.csv")
train_dataset = pd.read_csv(path_train, index_col = 0)
val_dataset = pd.read_csv(path_val, index_col = 0)
test_dataset = pd.read_csv(path_test, index_col = 0)

# Vocabulary

In [ ]:
train_dataset['kr_tokens'].head()

0    ['<sos>', "'", 'Bible', 'Coloring', "'", '은', ...
1    ['<sos>', '씨티', '은행', '에서', '일하', '세요', '?', '...
2    ['<sos>', '푸', '리', '토의', '베스트셀러', '는', '해외', ...
3    ['<sos>', '11', '장', '에서', '는', '예수', '님', '이'...
4    ['<sos>', '6.5', ',', '7', ',', '8', '사이즈', '가...
Name: kr_tokens, dtype: object

In [77]:
#각 행별로, 리스트로 반환?
train_dataset['en_tokens'] = train_dataset['en_tokens'].apply(literal_eval)
train_dataset['kr_tokens'] = train_dataset['kr_tokens'].apply(literal_eval)

val_dataset['en_tokens'] = val_dataset['en_tokens'].apply(literal_eval)
val_dataset['kr_tokens'] = val_dataset['kr_tokens'].apply(literal_eval)

test_dataset['en_tokens'] = test_dataset['en_tokens'].apply(literal_eval)
test_dataset['kr_tokens'] = test_dataset['kr_tokens'].apply(literal_eval)

train_dict = train_dataset.to_dict(orient = 'records')
val_dict = val_dataset.to_dict(orient = 'records')
test_dict = test_dataset.to_dict(orient = 'records')

In [78]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"
sos_token = "<sos>"
eos_token = "<eos>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

In [79]:
#훈련데이터에 있는 영어와 한국어 토큰을 바탕으로 어휘집을 생성하고, 어휘집에서 찾을 수 없는 토큰들을 처리하기 위한 방법을 설정
#텍스트를 숫자 형태로 변형
#고차원의 텍스트 데이터를, 차원축소하여 저장
#어휘집을 사용하여 훈련 데이터와 테스트 데이터를 동일한 방식으로 처리
#알 수 없는 토큰을 'unk'로 일관된 방식으로 처리
#sos나 eos, padding토큰같이 특후 토큰을 어휘집에 포함시켜 텍스트의 시작과 끝을 명확히하고, 동일한 길이로 맞춤
#
def yield_tokens(data, token_field):
    for item in data:
        yield item[token_field]

en_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "en_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

kr_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "kr_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

en_vocab.set_default_index(en_vocab[unk_token])
kr_vocab.set_default_index(kr_vocab[unk_token])

In [80]:
vocab_size_en = len(en_vocab)
vocab_size_kr = len(kr_vocab)
print("English Vocabulary Length : ", vocab_size_en)
print("Korean Vocabulary Length : ", vocab_size_kr)

English Vocabulary Length :  9772
Korean Vocabulary Length :  10442


# 문제 2 (Vocabulary Matching)

토큰화된 단어들을 Vocabulary 과 매칭 시켜서 **벡터**로 바꿔준 뒤에 **텐서**로 바꿔주는 단계까지 포함되어야 합니다!

힌트 :

- $\texttt{en_vocab.lookup_indices}$, 혹은 $\texttt{kr_vocab.lookup_indices}$
- 한줄로 표현이 가능합니다!
- en_indices 와 kr_indices 는 리스트이며, 텐서화 된 것들을 각 토큰마다 리스트로 가져야 합니다!

In [81]:
train_dataset['en_tokens']

0        [<sos>, bible, coloring, ', is, a, coloring, a...
1        [<sos>, do, you, work, at, a, city, bank, ?, <...
2        [<sos>, purito, 's, bestseller, ,, which, reco...
3        [<sos>, in, chapter, 11, jesus, called, lazaru...
4        [<sos>, i, would, feel, grateful, to, know, ho...
                               ...                        
29995    [<sos>, he, looks, shocked, and, does, n't, ha...
29996    [<sos>, he, was, thoroughly, criticized, and, ...
29997    [<sos>, he, is, living, in, seomun, -, dong, ,...
29998    [<sos>, does, n't, he, have, any, hobbies, ?, ...
29999    [<sos>, he, was, carried, to, the, hospital, f...
Name: en_tokens, Length: 30000, dtype: object

In [89]:
en_indices = []
for word in train_dataset['en_tokens']:
  en_indices.append(torch.tensor([en_vocab[w] for w in word]))

train_dataset['en_indices'] = en_indices

kr_indices = []
for word in train_dataset['kr_tokens']:
  kr_indices.append(torch.tensor([kr_vocab[w] for w in word]))

train_dataset['kr_indices'] = kr_indices

In [90]:
en_indices = []
for word in val_dataset['en_tokens']:
  en_indices.append(torch.tensor([en_vocab[w] for w in word]))

val_dataset['en_indices'] = en_indices

kr_indices = []
for word in val_dataset['kr_tokens']:
  kr_indices.append(torch.tensor([kr_vocab[w] for w in word]))

val_dataset['kr_indices'] = kr_indices

In [91]:
en_indices = []
for word in test_dataset['en_tokens']:
  en_indices.append(torch.tensor([en_vocab[w] for w in word]))


test_dataset['en_indices'] = en_indices

kr_indices = []
for word in test_dataset['kr_tokens']:
  kr_indices.append(torch.tensor([kr_vocab[w] for w in word]))

test_dataset['kr_indices'] = kr_indices

In [ ]:
train_dataset

#즉 위 과정은, 각 토큰들이 어휘집에서 몇번째 인덱스인가를 보여줌!

원문  \
0      'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...   
1                                           씨티은행에서 일하세요?   
2                  푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.   
3       11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.   
4         6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.   
...                                                  ...   
29995                             그는 충격받았는지 아무 표정이 없습니다.   
29996                      그는 충분히 비난받았고, 이제 내가 비난받을 차례야.   
29997                        그는 충청북도 청주시 상당구 서문동에 살고 있다.   
29998                                   그는 취미가 아무것도 없어요?   
29999                            그는 치료를 위해 병원으로 후송되었습니다.   

                                                     번역문  \
0      Bible Coloring' is a coloring application that...   
1                            Do you work at a City bank?   
2      PURITO's bestseller, which recorded 4th rough ...   
3      In Chapter 11 Jesus called Lazarus from the to...   
4      I would feel grateful to know how many stocks ...   
...                                                  ...   
29995  He looks shocked and doesn't have any facial e...   
29996   He was thoroughly criticized and now is my turn.   
29997  He is living in Seomun-dong, Sangdang-gu, Cheo...   
29998                       Doesn't he have any hobbies?   
29999      He was carried to the hospital for treatment.   

                                               en_tokens  \
0      [<sos>, bible, coloring, ', is, a, coloring, a...   
1      [<sos>, do, you, work, at, a, city, bank, ?, <...   
2      [<sos>, purito, 's, bestseller, ,, which, reco...   
3      [<sos>, in, chapter, 11, jesus, called, lazaru...   
4      [<sos>, i, would, feel, grateful, to, know, ho...   
...                                                  ...   
29995  [<sos>, he, looks, shocked, and, does, n't, ha...   
29996  [<sos>, he, was, thoroughly, criticized, and, ...   
29997  [<sos>, he, is, living, in, seomun, -, dong, ,...   
29998  [<sos>, does, n't, he, have, any, hobbies, ?, ...   
29999  [<sos>, he, was, carried, to, the, hospital, f...   

                                               kr_tokens  \
0      [<sos>, ', Bible, Coloring, ', 은, 성경, 의, 아름, 답...   
1                  [<sos>, 씨티, 은행, 에서, 일하, 세요, ?, <eos>]   
2      [<sos>, 푸, 리, 토의, 베스트셀러, 는, 해외, 에서, 입소문, 만, 으로...   
3      [<sos>, 11, 장, 에서, 는, 예수, 님, 이, 이번, 에, 는, 나사로,...   
4      [<sos>, 6.5, ,, 7, ,, 8, 사이즈, 가, 몇, 개, 나, 더, 재...   
...                                                  ...   
29995  [<sos>, 그, 는, 충격, 받, 았, 는지, 아무, 표정, 이, 없, 습니다,...   
29996  [<sos>, 그, 는, 충분히, 비난, 받, 았, 고, ,, 이제, 내가, 비난,...   
29997  [<sos>, 그, 는, 충청북도, 청주시, 상당구, 서문동, 에, 살, 고, 있,...   
29998      [<sos>, 그, 는, 취미, 가, 아무것, 도, 없, 어요, ?, <eos>]   
29999  [<sos>, 그, 는, 치료, 를, 위하, 어, 병원, 으로, 후송, 되, 었, ...   

                                              en_indices  \
0      [2, 3577, 5094, 91, 8, 7, 5094, 699, 17, 2654,...   
1               [2, 44, 16, 100, 37, 7, 475, 620, 20, 3]   
2      [2, 3777, 21, 0, 9, 76, 3182, 1092, 1734, 0, 4...   
3      [2, 14, 2665, 1145, 2723, 240, 0, 42, 5, 5675,...   
4      [2, 10, 62, 194, 2411, 6, 70, 74, 94, 2786, 26...   
...                                                  ...   
29995  [2, 15, 311, 3197, 12, 63, 34, 33, 115, 3646, ...   
29996  [2, 15, 23, 2328, 7757, 12, 144, 8, 40, 781, 4...   
29997  [2, 15, 8, 525, 14, 0, 35, 1076, 9, 0, 35, 149...   
29998              [2, 63, 34, 15, 33, 115, 6365, 20, 3]   
29999       [2, 15, 23, 1481, 6, 5, 510, 19, 1251, 4, 3]   

                                              kr_indices  
0      [2, 139, 0, 0, 139, 13, 5599, 17, 3240, 1020, ...  
1                     [2, 6864, 752, 28, 337, 55, 29, 3]  
2      [2, 1431, 354, 5937, 5521, 6, 1077, 28, 0, 70,...  
3      [2, 368, 459, 28, 6, 2853, 253, 7, 350, 9, 6, ...  
4      [2, 0, 27, 203, 27, 244, 674, 20, 255, 102, 47...  
...                                                  ...  
29995  [2, 10, 6, 2118, 74, 48, 155,

# Data Loaders

In [92]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item = self.dataframe.iloc[idx]
        en_indices = item['en_indices']
        kr_indices = item['kr_indices']
        return en_indices, kr_indices

In [93]:
batch_no = 80


#배치 데이터를 처리하여 모델에 공급할 수 있는 형태로 만듦
def collate_fn(batch):
    en_indices, kr_indices = zip(*batch)
    en_indices_padded = pad_sequence(en_indices, batch_first=True, padding_value=0) #모든 시퀀스가 같은 길이가 되도록 만듦
    kr_indices_padded = pad_sequence(kr_indices, batch_first=True, padding_value=0)

    return en_indices_padded, kr_indices_padded

#CustomDataset 인스턴스 생성, 각 인스턴스는 해당 데이터셋의 영어 및 한국어 인덱스만 포함하는 pandas 데이터프레임을 받음
train_custom_dataset = CustomDataset(train_dataset[['en_indices', 'kr_indices']])
val_custom_dataset = CustomDataset(val_dataset[['en_indices', 'kr_indices']])
test_custom_dataset = CustomDataset(test_dataset[['en_indices', 'kr_indices']])

#DataLoader는 이 CustomDataset을 사용하여 실제로 데이터를 모델에 공급할 배치를 생성.
train_loader = DataLoader(train_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)

In [94]:
#코드의 주요 목적은 각 배치 내에서 다양한 길이의 시퀀스를 효과적으로 처리하는 것입니다.
#여기서 사용되는 방법은 데이터 로더의 collate_fn 함수를 사용자 정의하는 것입니다.
def get_collate_fn(pad_index):
    def collate_fn(batch): #내부에서 pad_index를 적용하여 활용..! 외부의 함수는 변화 없음.
        batch_en_ids = [example["en_indices"] for example in batch]
        batch_de_ids = [example["de_indices"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

### 잘 매칭이 되었는지 확인해보기!

In [95]:
for i, (en_indices, kr_indices) in enumerate(train_loader):
    print(f"Batch {i+1}")
    print(f"Shape of en_indices: {en_indices.shape}")
    print(f"Shape of kr_indices: {kr_indices.shape}")

    print("Sample en_indices batch:", en_indices[:1])
    print("Sample kr_indices batch:", kr_indices[:1])

    if i == 1:
        break

Batch 1
Shape of en_indices: torch.Size([80, 29])
Shape of kr_indices: torch.Size([80, 37])
Sample en_indices batch: tensor([[   2,   29,   28,  451,   11, 4947,   39,   16,  614,  191,    5, 9657,
           82,    7,  305,    4,    3,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]])
Sample kr_indices batch: tensor([[   2,  663,  802,   16,  468, 2248,   14,   52,   54, 2175,    5,   23,
           35,   19,    6,  925,    7,  123,   19,   15,    4,    3,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0]])
Batch 2
Shape of en_indices: torch.Size([80, 28])
Shape of kr_indices: torch.Size([80, 33])
Sample en_indices batch: tensor([[   2,   10,   89,    5,    0, 1362,  570,   12, 4843,  615,    9,   49,
          125,    5,  599,  461,    4,    3,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])
Sample kr_indices batch: tensor([[   2,    0,    0, 5175,  222, 6776,  176,   52,

# 문제 3 (Modules without Attention!)

기본적인 Seq2seq 모듈들은 Teacher Forcing 까지 포함해서 진행해주시길 바랍니다.

In [96]:
class Encoder(nn.Module):
  """
  Encoder 내에서는 Input Language 의 Vocab 크기, Embedding 크기, Hidden 크기가 들어갑니다.

  Embedding 과정에서 Input 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Hidden Size]
  Hidden : [1, Batch_size, Hidden Size]

  힌트 :
  - Batch 가 앞에 있기 때문에 GRU 는 batch_first = True 라는 파라미터를 설정해줘야 합니다.
  - GRU 에서는 Output, Hidden Vector, 총 2개를 뱉어줍니다.
  - GRU 는 nn.GRU, Embedder 는 nn.Embedding 를 사용해주면 됩니다.
  """
  def __init__(self, input_vocab_size, embedding_size, hidden_size):
      super(Encoder, self).__init__()
      self.hidden_size = hidden_size
      # Embedding layer: Converts input vocab indices into dense representations
      self.embedding = nn.Embedding(num_embeddings=input_vocab_size, embedding_dim=embedding_size)

      # GRU layer: Processes the embedded input sequence into hidden states
      self.gru = nn.GRU(input_size=embedding_size, hidden_size=hidden_size, batch_first=True)





  def forward(self, input):
        # input_seq: [Batch_size, Input Sequences Max Size]

        # Convert input sequence to embeddings
        embedded = self.embedding(input)  # [Batch_size, Input Sequences Max Size, Embedding Size]

        # Process embeddings through GRU
        output, hidden = self.gru(embedded)  # output: [Batch_size, Input Sequences Max Size, Hidden Size]
                                             # hidden: [1, Batch_size, Hidden Size]

        return output, hidden



In [104]:
class Decoder(nn.Module):
  """
  Decoder 내에서는 Hidden 크기, Embedding Decoder 크기 (Embedding Encoder 크기와 동일), Output Language 의 Vocab 의 크기가 들어갑니다.

  Embedding 과정에서 Output 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.
  Linear 내에서는 Hidden 크기 --> Output 크기 로 진행됩니다.

  Teacher Forcing 은 디코더를 구성하는 코드에서는 고려를 안해도 되며, 나중에 Seq2Seq 모듈에서 진행해주시길 바랍니다.

  Input 과 Hidden 크기의 경우 Seq2Seq 모듈 내에서 차원을 맞춰주시면 됩니다.
  """
  def __init__(self, hidden_size, embedding_decoder_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        # Embedding layer for the decoder
        self.embedding = nn.Embedding(output_size, embedding_decoder_size)

        # GRU layer for the decoder
        self.gru = nn.GRU(embedding_decoder_size, hidden_size, batch_first=True)

        # Linear layer to map the hidden state output of the GRU to the output vocabulary size
        self.out = nn.Linear(hidden_size, output_size)
        self.output_size = output_size

  def forward(self, input, hidden):
      # Embed the input word indices
      embedded = self.embedding(input)  # Shape: [batch_size, 1, embedding_decoder_size]
      embedded = F.relu(embedded)

      # Pass the embedded input words and the initial hidden state to the GRU
      output, hidden = self.gru(embedded, hidden)  # Shape of output: [batch_size, 1, hidden_size]
                                                    # Shape of hidden: [1, batch_size, hidden_size]

      # Pass the GRU output through the linear layer to get our predicted word indices
      preds = self.out(output.squeeze(1))  # Shape: [batch_size, output_size]

      return preds, hidden

In [109]:
class Seq2Seq(nn.Module):
  """
  Seq2Seq 내에서는 Encoder, Decoder, Device 가 들어갑니다.

  Init 단계에서는 정말로 할 것이 크게 없으며, forward 단계에서 for loop 과 if 조건문들을 활용해서 Teacher Forcing 을 적용시켜주면 됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Target : [Batch_size, Target Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Input Language Vocab Size]

  힌트 :
  - outputs 란 0 으로 채워진 텐서를 하나 만들며 디코더의 t-번째 아웃풀을 outputs 의 t-번째 칸에 넣으면서 for loop 을 진행해주시면 됩니다.
  - Target 의 첫번째 요소 (:, 0) 는 항상 <sos> 입니다! 이것을 디코더의 첫 Input 으로 넣어준 다음에 for loop 을 진행해주시면 됩니다.
  - random.random() < teacher_forcing_ratio 를 통해서 조건문을 만들 수가 있습니다!
  - Teacher Force 가 적용이 안될 때는디코더의 아웃풋에는 모든 토큰들에 대한 점수가 나오게 됩니다.
    이것들 중에서 가장 최댓값만 중요하기에 argmax 를 이용해서 최댓값의 인덱스를 다음 Decoder 의 Input 으로 넣어주면 됩니다.
  - squeeze 와 unsqueeze 를 적극적으로 활용해서 차원을 조정해주시면 됩니다!
  """
  def __init__(self, encoder, decoder, device):
    super(Seq2Seq, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device





  def forward(self, input, target, teacher_forcing_ratio = 0.5):
    batch_size = target.shape[0]
    target_len = target.shape[1]
    target_vocab_size = self.decoder.output_size

    # Tensor to store decoder outputs
    outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)

    # Encode the input sequence
    encoder_output, hidden = self.encoder(input)

    # First input to the decoder is the <sos> tokens
    input = target[:, 0]

    for t in range(1, target_len):
        # Insert input token embedding, previous hidden state and obtain outputs
        output, hidden = self.decoder(input.unsqueeze(1), hidden)

        # Place predictions in a tensor holding predictions for each token
        outputs[:, t] = output

        # Decide whether to use teacher forcing or not
        teacher_force = random.random() < teacher_forcing_ratio

        # Get the highest predicted token from our predictions
        top1 = output.argmax(1)

        # If teacher forcing, use actual next token as next input; if not, use predicted token
        input = target[:, t] if teacher_force else top1

    return outputs

# Training

In [110]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder, device).to(device)

### 모델 확인 + Initialize 시키기

In [111]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 30,502,090 trainable parameters


### 하이퍼 파라미터 설정!

In [112]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

## Training 코드!

In [113]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss

    model.eval()
    epoch_loss = 0
    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()

    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

Epoch: 1, Train Loss : 2.797963171641032, Val Loss: 3.012195912618486
Epoch: 2, Train Loss : 2.4860454444885254, Val Loss: 3.0104647061181447
Epoch: 3, Train Loss : 2.311090971946716, Val Loss: 2.8794076102120534
Epoch: 4, Train Loss : 2.1554946231842043, Val Loss: 2.8775685476878334
Epoch: 5, Train Loss : 2.016433679262797, Val Loss: 2.766808963957287
Epoch: 6, Train Loss : 1.8605601342519125, Val Loss: 2.861301414550297
Epoch: 7, Train Loss : 1.7042867736816407, Val Loss: 2.9345985405028814
Epoch: 8, Train Loss : 1.5623134724299113, Val Loss: 2.9114117357465954
Epoch: 9, Train Loss : 1.4359006042480469, Val Loss: 3.0301271847316196
Epoch: 10, Train Loss : 1.3317252786954243, Val Loss: 2.9995656656840493


파라미터 저장

In [114]:
path_to_save = os.path.join(path_to_folder, "model_without_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [115]:
test_batch_en, test_batch_kr = next(iter(test_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', 'we', 'frequently', 'verify', 'that', 'customer', "'s", 'state', 'and', 'offer', 'help', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '그리', '고', '수시로', '고객', '의', '상태', '를', '확인', '하', '며', '도움', '을', '드리', 'ㅂ니다', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '와', '의', '및', '범위', '동의', '을', '하', '어야', 'ㅂ니다', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


\<unk\> 는 저희 Vocabulary 에는 존재하지 않는 토큰들을 의미합니다.

아무리 epoch 를 늘리거나 무엇을 해보려고 해도 완전히 좋은 결과가 나오지는 않을 것입니다. 해당 이유는 Attention 이 적용이 안되었기 때문이며, 다음과 같이 Attention 을 적용시켜서 한번 진행하면 결과가 다르다는 것을 알 수가 있습니다.

# 문제 4 (Modules WITH Attention)

만약에 메모리가 다 찼다면 해당 Run 을 Reconnect 시켜서 위의 Output 만 남긴 채로 제출하시면 됩니다.

**이렇게 진행하게 된다면 당연히 문제 3 이전의 코드들은 다시 돌려야 됩니다!!**

In [136]:
class Encoder(nn.Module):
  """
  이전과 Encoder 부분은 달라지는게 없습니다!
  """
  def __init__(self, input_size, embedding_encoder_size, hidden_size):
    super().__init__()
    self.embedding = nn.Embedding(input_size, embedding_encoder_size)
    self.gru = nn.GRU(embedding_encoder_size, hidden_size, batch_first=True)

  def forward(self, input):
    embedded = self.embedding(input)
    output, hidden = self.gru(embedded)
    return output, hidden


In [137]:
class Attention(nn.Module):
  """
  해당 모듈에서는 Attention Value 까지 찾는 단계입니다.

  Encoder 내에서의 Output 들의 개수마다 for loop 을 진행시켜서 합산시켜서 Attention Value 를 구하는 방법도 있지만,
  Encoder Output 들을 모두 Matrix 으로 표현해서 구하는 방법도 있습니다.

  Decoder_Outputs : [Batch_size, 1, Hidden_size]
  Encoder_Outputs : [Batch_size, Sequence Length, Hidden_size]

  힌트 :
  - Decoder_Outputs 의 차원이 위와 같이 안 나온다면 차원 조정을 위해서 squeeze, unsqueeze, transpose 등의 함수들을 활용해주면 됩니다.
  - Matrix Multiplication 을 진행할 때에 Batch 까지 곱해지는 것을 방지하기 위해서 torch.bmm(x, y) 를 활용해주면 됩니다.
  - 곱해줄 때에는 그저 x * y 를 해주면 됩니다.
  - 합산할 때에는 배치도 고려해야 함으로 torch.sum(x, dim = 1) 을 활용해주면 됩니다.
  - 해당 모듈 내에서 필요한 nn. 함수는 오로지 Softmax 밖에 없습니다.
  """

  def __init__(self, hidden_size):
    super(Attention, self).__init__()
    self.hidden_size = hidden_size

    self.attn_weights = nn.Linear(hidden_size * 2, 1) # input 으로 encoder, decoder 두 개를 받기 때문

  def forward(self, decoder_outputs, encoder_outputs):

    decoder_outputs = decoder_outputs.transpose(0, 1).transpose(1, 2) #결과 decoder_outputs =  [1, hidden_size, batch_size] * encdoer_outputs = [batch_size, seq_length, hidden_size]

    attn_scores = torch.bmm(encoder_outputs, decoder_outputs) # batch 빼고 계산

    attn_weights = F.softmax(attn_scores, dim=1)

    context_vector = torch.sum(attn_weights * encoder_outputs, dim=1)

    return context_vector, attn_weights


In [138]:
class Decoder(nn.Module):
  """
  Attention Value 를 Attention 모듈을 통해서 구했기 때문에 Concatenate, Tangent Hyperbolic 등의 것들이 여기에서 적용이 됩니다.

  힌트 :
  - 모든 것을 진행하기 전에 앞서서 먼저 GRU 를 지나고 나서 Output 을 활용해야 합니다.
  - Attention 모듈도 사용해야 하기 때문에 해당 모듈을 현 모듈에서 정의도 해줘야 합니다.
  - Concatenate 의 경우 torch.cat((x, y), dim = 1) 형태로 사용하면 됩니다.
  - W_c 와 v_t 를 곱해줄 때 W_c 는 가중치 행렬이며, 이것은 그저 nn.Linear 형태로 표현이 가능합니다.
  """
  def __init__(self, hidden_size, embedding_decoder_size, output_size):
        super().__init__()
        self.embedding = nn.Embedding(output_size, embedding_decoder_size)
        self.gru = nn.GRU(embedding_decoder_size + hidden_size, hidden_size, batch_first=True)
        self.attention = Attention(hidden_size)
        self.fc = nn.Linear(hidden_size * 2, output_size)

        self.output_size = output_size

  def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(1)  # [Batch_size, 1]
        embedded = self.embedding(input)  # [Batch_size, 1, Embedding_decoder_size]

        context_vector, attention_weights = self.attention(hidden, encoder_outputs)
        context_vector = context_vector.unsqueeze(1)  # [Batch_size, 1, Hidden_size]

        gru_input = torch.cat((embedded, context_vector), dim=2)  # [Batch_size, 1, Embedding_decoder_size + Hidden_size]
        output, hidden = self.gru(gru_input, hidden)  # output: [Batch_size, 1, Hidden_size], hidden: [1, Batch_size, Hidden_size]

        output = output.squeeze(1) #[Batch_size, Hidden_size]

        context_vector = context_vector.squeeze(1)  # [Batch_size, Hidden_size]

        combined_output = torch.cat((output, context_vector), dim=1)
        preds = self.fc(combined_output)

        return preds, hidden, attention_weights

In [139]:
class Seq2Seq(nn.Module):
  """
  이전과 Seq2Seq 부분은 크게 달라지는게 없습니다!
  단지 디코더가 Input 을 하나 더 받기 때문에 조심해주시길 바랍니다.
  """
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, input, target, teacher_forcing_ratio=0.5):
        batch_size = target.shape[0]
        target_seq_len = target.shape[1] #타겟 토큰 길이
        target_vocab_size = self.decoder.output_size # context vector의 차원

        # output을 저장할 tensor를 만듭니다.(처음에는 전부 0으로)
        outputs = torch.zeros(batch_size, target_seq_len, target_vocab_size).to(self.device)

        # input 문장을 encoder에 넣은 후 hidden, cell값을 구합니다.
        encoder_output, encoder_hidden = self.encoder(input)

        #decoder에 입력할 첫 번째 input
        #trg[:,0].shape = batch_size
        decoder_input = target[:, 0]  # 첫 번째 디코더 입력은 항상 <sos>

        for t in range(1, target_seq_len):
            # decoder에 입력한 첫 번째 input과 encoder_hidden을 이용하여 다음 step의 output 및 hidden state를 얻습니다.
            decoder_output, decoder_hidden, attn_weights = self.decoder(decoder_input, encoder_hidden, encoder_output)

            # prediction 저장
            outputs[:, t, :] = decoder_output

            # teacher forcing을 사용할지, 말지 결정
            teacher_force = random.random() < teacher_forcing_ratio

            # 가장 높은 확률을 갖은 값 얻기
            top1 = decoder_output.argmax(1)

            # teacher forcing의 경우에 다음 lstm에 target token 입력
            decoder_input = target[:, t] if teacher_force else top1

        return outputs

# Training

In [143]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder, device).to(device)

### 모델 확인 + Initialize 시키기

In [144]:
model.apply(init_weights)
print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 44,342,475 trainable parameters


### 하이퍼 파라미터 설정!

In [145]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

### 학습 시키기!

In [146]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()
    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

Epoch: 1, Train Loss : 3.0761583436330158, Val Loss: 3.6237784870087153
Epoch: 2, Train Loss : 2.7024389483133953, Val Loss: 3.6806091656760564
Epoch: 3, Train Loss : 2.470229078610738, Val Loss: 3.7151438925001354
Epoch: 4, Train Loss : 2.264378821372986, Val Loss: 3.756611055798001
Epoch: 5, Train Loss : 2.0773203048706055, Val Loss: 3.9672583549741716
Epoch: 6, Train Loss : 1.9623814687728882, Val Loss: 3.8855179945627847
Epoch: 7, Train Loss : 1.8828308944702148, Val Loss: 4.051366340546381
Epoch: 8, Train Loss : 1.786370366414388, Val Loss: 3.996259602289351
Epoch: 9, Train Loss : 1.727883557955424, Val Loss: 3.9245171887534007
Epoch: 10, Train Loss : 1.6804406226476034, Val Loss: 4.078170045973763


### 파라미터 저장

In [147]:
path_to_save = os.path.join(path_to_folder, "model_WITH_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [148]:
test_batch_en, test_batch_kr = next(iter(train_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', 'can', 'i', 'spend', 'more', 'time', 'with', '<unk>', 'after', '2', '?', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '2', '시', '이후', '에', '<unk>', '아', '와', '시간', '을', '더', '보내', 'ㄹ', '수', '있', '을까요', '?', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '시', '수', '있', '?', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


아마 결과가 온전히 잘 나오지는 않을 겁입니다. 해당 이유는 단순히 파라미터 개수가 적기 때문이며, 잘 나오기 위해서는 이보다 훨씬 늘려야 합니다. 원하신다면 Hidden_size, Embedding_size 를 늘려서 진행하는 것도 가능하긴 합니다!

과제 하시느라 정말로 고생하셨습니다!